In [ ]:
# default_exp voc2coco

# voc2coco

> Convert PASCAL VOX xml annotation to COCO json annotation.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export 
import os
import json
import re
import xml.etree.ElementTree as ET
from pathlib import Path
from typing import Dict, List
from tqdm import tqdm

In [ ]:
#export 
def get_image_info(annotation_root, img_id):
    filename = annotation_root.findtext('filename')
    img_name = os.path.basename(filename)

    size = annotation_root.find('size')
    width = int(size.findtext('width'))
    height = int(size.findtext('height'))

    image_info = {
        'file_name': filename,
        'height': height,
        'width': width,
        'id': img_id
    }
    return image_info

In [ ]:
#export
def get_coco_annotation_from_obj(obj, label2id):
    label = obj.findtext('name')
    assert label in label2id, f"Error: {label} is not in label2id !"
    category_id = label2id[label]
    bndbox = obj.find('bndbox')
    xmin = int(bndbox.findtext('xmin')) - 1
    ymin = int(bndbox.findtext('ymin')) - 1
    xmax = int(bndbox.findtext('xmax'))
    ymax = int(bndbox.findtext('ymax'))
    assert xmax > xmin and ymax > ymin, f"Box size error !: (xmin, ymin, xmax, ymax): {xmin, ymin, xmax, ymax}"
    o_width = xmax - xmin
    o_height = ymax - ymin
    ann = {
        'area': o_width * o_height,
        'iscrowd': 0,
        'bbox': [xmin, ymin, o_width, o_height],
        'category_id': category_id,
        'ignore': 0
    }
    return ann

In [ ]:
#export 
def convert_xmls_to_cocojson(annotation_paths: List[str],
                            label2id: Dict[str, int],
                            output_jsonpath: str):
    """Convert xml to coco json"""
    output_json_dict = {
        "images": [],
        "type": "instances",
        "annotations": [],
        "categories": []
    }
    bnd_id = 1  
    img_id = 1
    print('Start converting !')
    for a_path in tqdm(annotation_paths):
        # Read annotation xml
        ann_tree = ET.parse(a_path)
        ann_root = ann_tree.getroot()

        img_info = get_image_info(annotation_root=ann_root,
                                img_id=img_id)
        img_id = img_info['id']
        output_json_dict['images'].append(img_info)

        for obj in ann_root.findall('object'):
            ann = get_coco_annotation_from_obj(obj=obj, label2id=label2id)
            ann.update({'image_id': img_id, 'id': bnd_id})
            output_json_dict['annotations'].append(ann)
            bnd_id = bnd_id + 1
        img_id += 1

    for label, label_id in label2id.items():
        category_info = {'supercategory': 'none', 'id': label_id, 'name': label}
        output_json_dict['categories'].append(category_info)

    with open(output_jsonpath, 'w') as f:
        output_json = json.dumps(output_json_dict)
        f.write(output_json)

In [ ]:
show_doc(convert_xmls_to_cocojson)

<h4 id="convert_xmls_to_cocojson" class="doc_header"><code>convert_xmls_to_cocojson</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>convert_xmls_to_cocojson</code>(**`annotation_paths`**:`List`\[`str`\], **`label2id`**:`Dict`\[`str`, `int`\], **`output_jsonpath`**:`str`)

Convert xml to coco json